In [1]:
from pyspark.sql import SparkSession, functions as F
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
import os
from pyspark.sql.types import *

In [2]:
spark = (SparkSession.builder
.appName("Test1") 
.master("local[2]")
.config("spark.driver.memory","4g")
.getOrCreate())

2023-03-02 18:49:35,177 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.version

'3.1.1'

In [4]:
! wget -O ~/datasets/tmdb_5000_movies_and_credits.zip https://github.com/erkansirin78/datasets/raw/master/tmdb_5000_movies_and_credits.zip

--2023-03-02 18:49:44--  https://github.com/erkansirin78/datasets/raw/master/tmdb_5000_movies_and_credits.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/tmdb_5000_movies_and_credits.zip [following]
--2023-03-02 18:49:44--  https://raw.githubusercontent.com/erkansirin78/datasets/master/tmdb_5000_movies_and_credits.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8600683 (8.2M) [application/zip]
Saving to: ‘/home/train/datasets/tmdb_5000_movies_and_credits.zip’

100%[======================================>] 8,600,683   4.44MB/s   in 1.8s   

2023-03-02 18:49:48

In [5]:
! cd /home/train/datasets/; yes | unzip tmdb_5000_movies_and_credits.zip

Archive:  tmdb_5000_movies_and_credits.zip
   creating: tmdb_5000_movies_and_credits/
  inflating: tmdb_5000_movies_and_credits/tmdb_5000_credits.csv  
  inflating: tmdb_5000_movies_and_credits/tmdb_5000_movies.csv  
yes: standard output: Broken pipe
yes: write error


In [8]:
! ls -l /home/train/datasets/tmdb_5000_movies_and_credits

total 44676
-rw-rw-r--. 1 train train 40044293 Oct 20 08:43 tmdb_5000_credits.csv
-rw-rw-r--. 1 train train  5698599 Oct 20 08:43 tmdb_5000_movies.csv


# Read credits

In [9]:
credits = spark.read.format("csv") \
.option("header", True) \
.option("quote", "\"") \
.option("escape", "\"") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/tmdb_5000_movies_and_credits/tmdb_5000_credits.csv")

In [10]:
credits.write.mode("overwrite").save("file:///home/train/datasets/tmdb_5000_credits_parq")

In [11]:
credits = spark.read \
.load("file:///home/train/datasets/tmdb_5000_credits_parq")

In [12]:
credits.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)



In [13]:
credits.count()

4803

In [14]:
credits.columns

['movie_id', 'title', 'cast', 'crew']

In [15]:
cast_schema = StructType([
StructField("cast_id", IntegerType(), True),
StructField("character", StringType(), True),
StructField("credit_id", StringType(), True),
StructField("gender", IntegerType(), True),
StructField("id", IntegerType(), True),
StructField("name", StringType(), True)
])

In [16]:
credits1 = credits.withColumn("cast", F.from_json(F.col("cast"), ArrayType( cast_schema )))

In [17]:
credits1.select("crew").limit(1).toPandas()

crew
0  [{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"}, {"credit_id": "52fe48009251416c750ac9d9", "department": "Writing", "gender": 2, "id": 2710, "job": "Writer", "name": "James Cameron"}, {"credit_id": "52fe48009251416c750aca17", "department": "Editing", "gender": 2, "id": 2710, "job": "Editor", "name": "James Cameron"}, {"credit_id": "52fe48009251416c750aca29", "department": "Production", "gender": 2, "id": 2710, "job": "Producer", "name": "James Cameron"}, {"credit_id": "52fe48009251416c750aca3f", "department": "Writing", "gender": 2, "id": 2710, "job": "Screenplay", "name": "James Cameron"}, {"credit_id": "539c4987c3a36810ba0021a4", "department": "Art", "gender": 2, "id": 7236, "job": "Art Direction", "name": "Andrew Menzies"}, {"credit_id": "549598c3c3a3686ae9004383", "department": "Visual Effects", "gender": 0, "id": 6690, "job": "Visual Effects Producer", "name": "Jill Brooks"}, {"credit_id": "52fe48009251416c750aca4b", "department": "Production", "gender": 1, "id": 6347, "job": "Casting", "name": "Margery Simkin"}, {"credit_id": "570b6f419251417da70032fe", "department": "Art", "gender": 2, "id": 6878, "job": "Supervising Art Director", "name": "Kevin Ishioka"}, {"credit_id": "5495a0fac3a3686ae9004468", "department": "Sound", "gender": 0, "id": 6883, "job": "Music Editor", "name": "Dick Bernstein"}, {"credit_id": "54959706c3a3686af3003e81", "department": "Sound", "gender": 0, "id": 8159, "job": "Sound Effects Editor", "name": "Shannon Mills"}, {"credit_id": "54491d58c3a3680fb1001ccb", "department": "Sound", "gender": 0, "id": 8160, "job": "Foley", "name": "Dennie Thorpe"}, {"credit_id": "54491d6cc3a3680fa5001b2c", "department": "Sound", "gender": 0, "id": 8163, "job": "Foley", "name": "Jana Vance"}, {"credit_id": "52fe48009251416c750aca57", "department": "Costume & Make-Up", "gender": 1, "id": 8527, "job": "Costume Design", "name": "Deborah Lynn Scott"}, {"credit_id": "52fe48009251416c750aca2f", "department": "Production", "gender": 2, "id": 8529, "job": "Producer", "name": "Jon Landau"}, {"credit_id": "539c4937c3a36810ba002194", "department": "Art", "gender": 0, "id": 9618, "job": "Art Direction", "name": "Sean Haworth"}, {"credit_id": "539c49b6c3a36810c10020e6", "department": "Art", "gender": 1, "id": 12653, "job": "Set Decoration", "name": "Kim Sinclair"}, {"credit_id": "570b6f2f9251413a0e00020d", "department": "Art", "gender": 1, "id": 12653, "job": "Supervising Art Director", "name": "Kim Sinclair"}, {"credit_id": "54491a6c0e0a26748c001b19", "department": "Art", "gender": 2, "id": 14350, "job": "Set Designer", "name": "Richard F. Mays"}, {"credit_id": "56928cf4c3a3684cff0025c4", "department": "Production", "gender": 1, "id": 20294, "job": "Executive Producer", "name": "Laeta Kalogridis"}, {"credit_id": "52fe48009251416c750aca51", "department": "Costume & Make-Up", "gender": 0, "id": 17675, "job": "Costume Design", "name": "Mayes C. Rubeo"}, {"credit_id": "52fe48009251416c750aca11", "department": "Camera", "gender": 2, "id": 18265, "job": "Director of Photography", "name": "Mauro 

In [18]:
credits1.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- cast: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cast_id: integer (nullable = true)
 |    |    |-- character: string (nullable = true)
 |    |    |-- credit_id: string (nullable = true)
 |    |    |-- gender: integer (nullable = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- crew: string (nullable = true)



In [19]:
crew_schema = StructType([
StructField("credit_id", StringType(), True),
StructField("department", StringType(), True),
StructField("gender", IntegerType(), True),
StructField("id", IntegerType(), True),
StructField("job", StringType(), True),
StructField("name", StringType(), True)
])

In [20]:
credits2 = credits1.withColumn("crew", F.from_json(F.col("crew"), ArrayType( crew_schema )))

In [21]:
credits2.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- cast: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cast_id: integer (nullable = true)
 |    |    |-- character: string (nullable = true)
 |    |    |-- credit_id: string (nullable = true)
 |    |    |-- gender: integer (nullable = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- crew: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- credit_id: string (nullable = true)
 |    |    |-- department: string (nullable = true)
 |    |    |-- gender: integer (nullable = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- job: string (nullable = true)
 |    |    |-- name: string (nullable = true)



In [22]:
credits2.count()

4803

In [23]:
credits2.show(2)

+--------+--------------------+--------------------+--------------------+
|movie_id|               title|                cast|                crew|
+--------+--------------------+--------------------+--------------------+
|   19995|              Avatar|[{242, Jake Sully...|[{52fe48009251416...|
|     285|Pirates of the Ca...|[{4, Captain Jack...|[{52fe4232c3a3684...|
+--------+--------------------+--------------------+--------------------+
only showing top 2 rows



# Query

In [24]:
credits2.select("cast").limit(2).toPandas()

cast
0  [(242, Jake Sully, 5602a8a7c3a3685532001c9a, 2, 65731, Sam Worthington), (3, Neytiri, 52fe48009251416c750ac9cb, 1, 8691, Zoe Saldana), (25, Dr. Grace Augustine, 52fe48009251416c750aca39, 1, 10205, Sigourney Weaver), (4, Col. Quaritch, 52fe48009251416c750ac9cf, 2, 32747, Stephen Lang), (5, Trudy Chacon, 52fe48009251416c750ac9d3, 1, 17647, Michelle Rodriguez), (8, Selfridge, 52fe48009251416c750ac9e1, 2, 1771, Giovanni Ribisi), (7, Norm Spellman, 52fe48009251416c750ac9dd, 2, 59231, Joel David Moore), (9, Moat, 52fe48009251416c750ac9e5, 1, 30485, CCH Pounder), (11, Eytukan, 52fe48009251416c750ac9ed, 2, 15853, Wes Studi), (10, Tsu'Tey, 52fe48009251416c750ac9e9, 2, 10964, Laz Alonso), (12, Dr. Max Patel, 52fe48009251416c750ac9f1, 2, 95697, Dileep Rao), (13, Lyle Wainfleet, 52fe48009251416c750ac9f5, 2, 98215, Matt Gerald), (32, Private Fike, 52fe48009251416c750aca5b, 2, 154153, Sean Anthony Moran), (33, Cryo Vault Med Tech, 52fe48009251416c750aca5f, 2, 397312, Jason Whyte), (34, Venture Star Crew Chief, 52fe48009251416c750aca63, 2, 42317, Scott Lawrence), (35, Lock Up Trooper, 52fe48009251416c750aca67, 2, 986734, Kelly Kilgour), (36, Shuttle Pilot, 52fe48009251416c750aca6b, 0, 1207227, James Patrick Pitt), (37, Shuttle Co-Pilot, 52fe48009251416c750aca6f, 0, 1180936, Sean Patrick Murphy), (38, Shuttle Crew Chief, 52fe48009251416c750aca73, 2, 1019578, Peter Dillon), (39, Tractor Operator / Troupe, 52fe48009251416c750aca77, 0, 91443, Kevin Dorman), (40, Dragon Gunship Pilot, 52fe48009251416c750aca7b, 2, 173391, Kelson Henderson), (41, Dragon Gunship Gunner, 52fe48009251416c750aca7f, 0, 1207236, David Van Horn), (42, Dragon Gunship Navigator, 52fe48009251416c750aca83, 0, 215913, Jacob Tomuri), (43, Suit #1, 52fe48009251416c750aca87, 0, 143206, Michael Blain-Rozgay), (44, Suit #2, 52fe48009251416c750aca8b, 2, 169676, Jon Curry), (46, Ambient Room Tech, 52fe48009251416c750aca8f, 0, 1048610, Luke Hawker), (47, Ambient Room Tech / Troupe, 52fe48009251416c750aca93, 0, 42288, Woody Schultz), (48, Horse Clan Leader, 52fe48009251416c750aca97, 2, 68278, Peter Mensah), (49, Link Room Tech, 52fe48009251416c750aca9b, 0, 1207247, Sonia Yee), (50, Basketball Avatar / Troupe, 52fe48009251416c750aca9f, 1, 1207248, Jahnel Curfman), (51, Basketball Avatar, 52fe48009251416c750acaa3, 0, 89714, Ilram Choi), (52, Na'vi Child, 52fe48009251416c750acaa7, 0, 1207249, Kyla Warren), (53, Troupe, 52fe48009251416c750acaab, 0, 1207250, Lisa Roumain), (54, Troupe, 52fe48009251416c750acaaf, 1, 83105, Debra Wilson), (57, Troupe, 52fe48009251416c750acabb, 0, 1207253, Chris Mala), (55, Troupe, 52fe48009251416c750acab3, 0, 1207251, Taylor Kibby), (56, Troupe, 52fe48009251416c750acab7, 0, 1207252, Jodie Landau), (58, Troupe, 52fe48009251416c750acabf, 0, 1207254, Julie Lamm), (59, Troupe, 52fe48009251416c750acac3, 0, 1207257, Cullen B. Madden), (60, Troupe, 52fe48009251416c750acac7, 0, 1207259, Joseph Brady Madden), (61, Troupe, 52fe48009251416c750acacb, 0, 1207262, Frankie Torres), (62, Troupe, 52fe48009251416c750acacf, 1, 1158600, Austin Wilson), (63, Troupe, 52fe48019251416c750acad3, 1, 983705, Sara Wilson), (64, Troupe, 52fe48019251416c750acad7, 0, 1207263, Tamica Washington-Miller), (65, Op Center Staff, 52fe48019251416c750acadb, 1, 1145098, Lucy Briant), (66, Op Center Staff, 52fe48019251416c750acadf, 2, 33305, Nathan Meister), (67, Op Center Staff, 52fe48019251416c750acae3, 0, 1207264, Gerry Blair), (68, Op Center Staff, 52fe48019251416c750acae7, 2, 33311, Matthew Chamberlain), (69, Op Center Staff, 52fe48019251416c750acaeb, 0, 1207265, Paul Yates), (70, Op Center Duty Officer, 52fe48019251416c750acaef, 0, 1207266, Wray Wilson), (71, Op Center Staff, 52fe48019251416c750acaf3, 2, 54492, James Gaylyn), (72, Dancer, 52fe48019251416c750acaf7, 0, 1207267, Melvin Leno Clark III), (73, Dancer, 52fe48019251416c750acafb, 0, 1207268, Carvon Futrell), (74, Dancer, 52fe48019251416c750acaff, 0, 1207269, Brandon Jelkes), (75, Dancer, 52fe48019251416c750acb03, 0, 1207270, Mic

# Explode cast

In [25]:
credits_cast_expld = credits2.select("movie_id", "title", F.explode_outer("cast").alias("cast"))

In [26]:
credits_cast_expld.count()

106300

In [27]:
credits_cast_expld.select('cast').limit(1).toPandas()

,cast
0,"(242, Jake Sully, 5602a8a7c3a3685532001c9a, 2, 65731, Sam Worthington)"


In [28]:
credits_cast_expld.select("movie_id", "cast").limit(2).toPandas()

,movie_id,cast
0,19995,"(242, Jake Sully, 5602a8a7c3a3685532001c9a, 2, 65731, Sam Worthington)"
1,19995,"(3, Neytiri, 52fe48009251416c750ac9cb, 1, 8691, Zoe Saldana)"


In [29]:
cast_cols = ['cast_id', 'character', 'credit_id', 'gender', 'id', 'name']
cast_cols

['cast_id', 'character', 'credit_id', 'gender', 'id', 'name']

In [30]:
for col in cast_cols:
    credits_cast_expld = credits_cast_expld.withColumn(col, F.col("cast").getItem(col))

In [31]:
credits_cast = credits_cast_expld.drop('cast')

In [32]:
credits_cast.show(4, truncate=False)

+--------+------+-------+-------------------+------------------------+------+-----+----------------+
|movie_id|title |cast_id|character          |credit_id               |gender|id   |name            |
+--------+------+-------+-------------------+------------------------+------+-----+----------------+
|19995   |Avatar|242    |Jake Sully         |5602a8a7c3a3685532001c9a|2     |65731|Sam Worthington |
|19995   |Avatar|3      |Neytiri            |52fe48009251416c750ac9cb|1     |8691 |Zoe Saldana     |
|19995   |Avatar|25     |Dr. Grace Augustine|52fe48009251416c750aca39|1     |10205|Sigourney Weaver|
|19995   |Avatar|4      |Col. Quaritch      |52fe48009251416c750ac9cf|2     |32747|Stephen Lang    |
+--------+------+-------+-------------------+------------------------+------+-----+----------------+
only showing top 4 rows



In [33]:
credits_cast.count()

106300

In [34]:
credits_cast.select("credit_id").distinct().count()

106258

In [35]:
credits_cast.select("id").distinct().count()

54589

In [36]:
credits_cast.select("cast_id").distinct().count()

525

In [37]:
credits_cast.select("movie_id").distinct().count()

4803

In [38]:
credits_cast.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- cast_id: integer (nullable = true)
 |-- character: string (nullable = true)
 |-- credit_id: string (nullable = true)
 |-- gender: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



# Explode credit

In [39]:
credits_crew_expld = credits2.select("movie_id", "title", F.explode_outer("crew").alias("crew"))

In [40]:
credits_crew_expld.select('crew').limit(2).toPandas()

,crew
0,"(52fe48009251416c750aca23, Editing, 0, 1721, Editor, Stephen E. Rivkin)"
1,"(539c47ecc3a36810e3001f87, Art, 2, 496, Production Design, Rick Carter)"


In [41]:
credits_crew_expld.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- crew: struct (nullable = true)
 |    |-- credit_id: string (nullable = true)
 |    |-- department: string (nullable = true)
 |    |-- gender: integer (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- job: string (nullable = true)
 |    |-- name: string (nullable = true)



In [42]:
crew_cols = ['credit_id', 'department', 'gender', 'id', 'job', 'name']
crew_cols

['credit_id', 'department', 'gender', 'id', 'job', 'name']

In [43]:
for col in crew_cols:
    credits_crew_expld = credits_crew_expld.withColumn(col, F.col("crew").getItem(col))

In [44]:
credits_crew = credits_crew_expld.drop("crew")

In [45]:
credits_crew.show(4, truncate=False)

+--------+------+------------------------+----------+------+----+------------------------+-----------------+
|movie_id|title |credit_id               |department|gender|id  |job                     |name             |
+--------+------+------------------------+----------+------+----+------------------------+-----------------+
|19995   |Avatar|52fe48009251416c750aca23|Editing   |0     |1721|Editor                  |Stephen E. Rivkin|
|19995   |Avatar|539c47ecc3a36810e3001f87|Art       |2     |496 |Production Design       |Rick Carter      |
|19995   |Avatar|54491c89c3a3680fb4001cf7|Sound     |0     |900 |Sound Designer          |Christopher Boyes|
|19995   |Avatar|54491cb70e0a267480001bd0|Sound     |0     |900 |Supervising Sound Editor|Christopher Boyes|
+--------+------+------------------------+----------+------+----+------------------------+-----------------+
only showing top 4 rows



In [46]:
credits_crew.count()

129609

In [47]:
credits_crew.select('movie_id').distinct().count()

4803

In [48]:
credits_crew.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- credit_id: string (nullable = true)
 |-- department: string (nullable = true)
 |-- gender: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- name: string (nullable = true)



In [49]:
spark.stop()